In [1]:
import os.path
import mne
import numpy as np
import pandas as pd

from utils.dataloader import load_subject_labels

wd = '/Volumes/Guillaume EEG Project'
epoch_data_path = os.path.join(wd, 'Berlin_Data/EEG/preprocessed/stim_epochs')
raw_data_path = os.path.join(wd, 'Berlin_Data/EEG/raw')

In [2]:
# Load the raw data
subject_id = 101
raws = [mne.io.read_raw_brainvision(os.path.join(raw_data_path, '{}/proband_{}.vhdr'.format(subject_id, subject_id-100)))]

# raws = [mne.io.read_raw_brainvision(os.path.join(raw_data_path, '{}/{}.vhdr'.format(subject_id, subject_id)))]
# for file in [os.path.join(raw_data_path, '{}/{}_{}.vhdr'.format(subject_id, subject_id, suffix)) for suffix in range(1, 7)]:
#     if os.path.isfile(file):
#         raw = mne.io.read_raw_brainvision(file)
#         raws.append(raw)

behav_data = load_subject_labels(raw_data_path, subject_id=subject_id)

print(behav_data)

Extracting parameters from /Volumes/Guillaume EEG Project/Berlin_Data/EEG/raw/101/proband_1.vhdr...
Setting channel info structure...
     response  confidence  choice_rt  correct  session  run
0         1.0         1.0   0.905540      0.0        1    3
1         1.0         1.0   0.454888      1.0        1    3
2         1.0         1.0   0.943555      0.0        1    3
3         1.0         1.0   0.823302      1.0        1    3
4        -1.0         2.0   0.498823      1.0        1    3
..        ...         ...        ...      ...      ...  ...
325       1.0         1.0   0.801202      0.0        1    6
326       1.0         1.0   0.609246      0.0        1    6
327      -1.0         2.0   0.450043      1.0        1    6
328       1.0         1.0   0.592723      0.0        1    6
329       1.0         1.0   0.812557      1.0        1    6

[330 rows x 6 columns]


In [5]:
## SPLIT RAW INTO BLOCKS
# Specify the annotation type you want to use for splitting
split_annotation_type = 'Stimulus/S105'

# Adjust events: keep only events within the cropped range
# TODO: compare method below with pyEEG.preprocessing.ant2time_window !!!
#   e.g.: do I have to divide start_sample and stop_sample with raw.info['sfreq']?
# OTHER OPTION: can't I just modify annotations, without first converting to events?
def adjust_events(raw, events):
    """ Keep only events within the range of the raw data """
    start_sample = int(raw.first_samp)  # First sample of cropped data
    stop_sample = int(raw.last_samp)  # Last sample of cropped data
    valid_events = events[(events[:, 0] >= start_sample) & (events[:, 0] <= stop_sample)]

    # Shift event onset times relative to new start
    valid_events[:, 0] -= start_sample
    return valid_events


raw_segments = []
for raw in raws:
    # Find the onset times of the relevant annotations
    split_onsets = [annot['onset']-1 for annot in raw.annotations if annot['description'] == split_annotation_type]

    # Sort the onsets and add the start and end times of the raw object for splitting
    split_onsets = split_onsets + [raw.times[-1]]

    events, event_id = mne.events_from_annotations(raw)

    # Iterate over consecutive pairs of split points to create new raw segments
    for start, end in zip(split_onsets[:-1], split_onsets[1:]):
        raw_segment = raw.copy().crop(tmin=start, tmax=end, include_tmax=False)

        adjusted_events = adjust_events(raw_segment, events)
        adjusted_annotations = mne.annotations_from_events(adjusted_events, raw_segment.info['sfreq'], event_desc=dict((v, k) for k, v in event_id.items()))
        raw_segment.set_annotations(adjusted_annotations)

        # trial_n = pd.Series(raw_segment.annotations.description).value_counts()['Stimulus/S151']
        # if trial_n == 55:
        raw_segments.append(raw_segment)


raw_segments = raw_segments[1:]
# Now raw_segments is a list of mne.io.Raw objects split at the specified annotations
print(len(raw_segments))

for i, seg in enumerate(raw_segments):
    print('---------------{}---------------'.format(i+1))
    print(seg)
    # print(pd.Series(seg.annotations.description).value_counts())
    trial_n = pd.Series(seg.annotations.description).value_counts()['Stimulus/S151']
    print(seg.info['meas_date'])
    print('{} trials'.format(trial_n))
    print('')

    # mne.export.export_raw("segment{}.vhdr".format(i), seg, fmt="brainvision", overwrite=True)

Used Annotations descriptions: [np.str_('New Segment/'), np.str_('Stimulus/S  3'), np.str_('Stimulus/S  4'), np.str_('Stimulus/S  5'), np.str_('Stimulus/S  6'), np.str_('Stimulus/S  7'), np.str_('Stimulus/S  8'), np.str_('Stimulus/S 10'), np.str_('Stimulus/S 11'), np.str_('Stimulus/S 12'), np.str_('Stimulus/S 20'), np.str_('Stimulus/S 21'), np.str_('Stimulus/S 22'), np.str_('Stimulus/S 23'), np.str_('Stimulus/S 25'), np.str_('Stimulus/S 28'), np.str_('Stimulus/S 29'), np.str_('Stimulus/S 40'), np.str_('Stimulus/S 41'), np.str_('Stimulus/S 46'), np.str_('Stimulus/S 48'), np.str_('Stimulus/S 50'), np.str_('Stimulus/S 56'), np.str_('Stimulus/S 64'), np.str_('Stimulus/S 73'), np.str_('Stimulus/S 76'), np.str_('Stimulus/S 88'), np.str_('Stimulus/S105'), np.str_('Stimulus/S150'), np.str_('Stimulus/S151')]
7
---------------1---------------
<RawBrainVision | proband_1.eeg, 63 x 92803 (92.8 s), ~93 kB, data not loaded>
2022-01-17 11:15:05.598816+00:00
3 trials

---------------2---------------
<

In [6]:
# a_raw = raw_segments[0]
# ann = a_raw.annotations.copy()
# ann.onset -= a_raw.first_samp / a_raw.info['sfreq']
# a_raw.set_annotations(ann)

In [5]:
raw_segments_compare = [mne.io.read_raw_brainvision("segment{}.vhdr".format(i)) for i in range(len(raw_segments))]
pass

Extracting parameters from segment0.vhdr...
Setting channel info structure...
Extracting parameters from segment1.vhdr...
Setting channel info structure...
Extracting parameters from segment2.vhdr...
Setting channel info structure...
Extracting parameters from segment3.vhdr...
Setting channel info structure...
Extracting parameters from segment4.vhdr...
Setting channel info structure...
Extracting parameters from segment5.vhdr...
Setting channel info structure...
Extracting parameters from segment6.vhdr...
Setting channel info structure...
Extracting parameters from segment7.vhdr...
Setting channel info structure...


KeyboardInterrupt: 

In [ ]:
## CREATE NEW ANNOTATIONS FOR RESPONSE TIMES
for i, seg in enumerate(raw_segments):
    df = behav_data[behav_data['run'] == i + 1]

    # Step 1: Extract the onsets of 'Stimulus/S 48' annotations
    stimulus_onsets = [annot['onset'] for annot in seg.annotations if annot['description'] == 'Stimulus/S 48']
    test1 = [annot['onset'] for annot in seg.annotations if annot['description'] == 'Stimulus/S150']
    test2 = [annot['onset'] for annot in seg.annotations if annot['description'] == 'Stimulus/S151']
    test3 = [annot['onset'] for annot in seg.annotations if annot['description'] == 'Stimulus/S 64']

    i = 0
    for a,b,c,d in zip(test1, test3, stimulus_onsets, test2):
        print(a, b-a, c-a, d-a)
        i += 1
        if i == 5:
            break


    # Step 2: Ensure the number of onsets matches the number of rows in the DataFrame
    if len(stimulus_onsets) != len(df):
        raise ValueError("The number of 'Stimulus/S 48' ({}) annotations does not match the number of rows in the DataFrame ({}).".format(len(stimulus_onsets), len(df)))

    # Step 3: Create new annotations based on the reaction times in the DataFrame
    for onset, choice_rt in zip(stimulus_onsets, df['choice_rt']):
        late_threshold = 1.000
        if not np.isnan(choice_rt):
            new_onset = onset + choice_rt  # Calculate the new annotation onset time
            if choice_rt < late_threshold:
                seg.annotations.append(onset=new_onset, duration=0, description='Response')
            else:
                seg.annotations.append(onset=new_onset, duration=0, description='Late Response')
        else:
            new_onset = onset + 0.001  # Calculate the new annotation onset time
            seg.annotations.append(onset=new_onset, duration=0, description='No Response')

In [ ]:
# for i, an in enumerate(raw_segments[-1].annotations):
#     if an['description'] in [*['Stimulus/S{:>3}'.format(stim) for stim in [20, 21, 22, 23, 48]], 'Response', 'No Response']:
#         print(an)
#     if i == 200:
#         break

In [ ]:
## EXTRACT EPOCHS
# Define the epochs
tmin = -1.0  # Start of each epoch (e.g., 2250 ms before the event)
tmax = 2.5   # End of each epoch (e.g., 250 ms after the event)

epochs_list = []
for seg in raw_segments:
    events, event_id = mne.events_from_annotations(seg, event_id={'Stimulus/S 64': 1}) #event_id={'Response': 1, 'Late Response': 2, 'No Response': 3})
    epochs_list.append(mne.Epochs(seg, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True))

# Step 5: Inspect the epochs
print(epochs_list[0])

In [ ]:
# TODO: select epochs that have response, drop the rest according to annotation 'Late Response' / 'No Response'.

In [ ]:
raw_compare = mne.io.read_raw_brainvision("first_half.vhdr")
pass